# Lib

In [29]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import warnings

## ignore warning log in pandas
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import sys
import json

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")

from utils import utils_ml, parameters, metrics_evaluation, Pipeline_functions

from ipywidgets import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, make_scorer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

29/11/2022  02:35    <DIR>          .
23/11/2022  00:25    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
29/11/2022  02:35         1.008.994 1.EA.ipynb
25/11/2022  01:27            74.828 2.Feature_engineering.ipynb
29/11/2022  03:09            28.642 3.Stability_var.ipynb
04/12/2022  14:59           170.765 4.Modeling.ipynb
29/11/2022  02:48            10.342 5.Modelo_analisis_negocio.ipynb
25/11/2022  01:32           156.224 6.Pipeline.ipynb
25/11/2022  01:32           156.224 7.Pipeline_SPARK.ipynb
24/11/2022  23:44    <DIR>          catboost_info
               7 arquivo(s)      1.606.019 bytes
               4 pasta(s)   364.138.418.176 bytes dispon�veis


# Reading

In [31]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [32]:
!dir .\\..\data

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\data

04/12/2022  11:04    <DIR>          .
23/11/2022  00:25    <DIR>          ..
04/12/2022  11:04            24.658 cat_dataset.csv
10/08/2021  12:08        26.677.890 dados.csv
28/11/2022  00:10        14.104.434 dataset_structed.parquet
28/11/2022  00:05        27.547.133 EA.pbix
22/11/2022  22:34            39.018 iv_woe_by_week.parquet
09/11/2022  10:35            54.091 MySQL_localhost_connection.pbix
24/11/2022  23:31         3.891.056 test_dataset.parquet
24/11/2022  23:31         8.711.592 train_dataset.parquet
25/11/2022  01:12        14.493.356 train_test_dataset_tranformed.parquet
24/11/2022  23:31         2.094.822 validation_dataset.parquet
              10 arquivo(s)     97.638.050 bytes
               2 pasta(s)   364.140.482.560 bytes dispon�veis


In [67]:
def create_woe_columns(df):

    ## criando uma copia
    dff = df.copy()
    ## Selecionando as colunas para serem usadas
    colunas_orig_and_week = utils_ml.exclude_words( dff.columns.tolist(), ["cat","bin","WoE","produto","fecha","data","hora","week","fraude"] ) + ['categoria_produto']
    ## Iterando para adicionar as colunas WoE
    for n,variable in enumerate(colunas_orig_and_week, start=1):

        optb = joblib.load(f".\..\src\\features\\pipeline_binning__{variable}__.pkl")

        dff = utils_ml.encode_woe_var_transform(dff, variable, optb )


    return dff


In [34]:
## test-train dataset
df = pd.read_parquet(f"{parameters.data_path}\\train_test_dataset_tranformed.parquet")
df['data'] = pd.to_datetime(df['data'])

## validation Dataset
df_validation = pd.read_parquet(f"{parameters.data_path}\\validation_dataset.parquet")
df_validation['data'] = pd.to_datetime(df_validation['data'])
## Creating WoE columns
df_validation = create_woe_columns(df_validation)

In [35]:
features_bins_woe = utils_ml.list_subset_words( df.columns.tolist(), ["bin"])
## Excluding variable 'score' (score of Meli model) and 'o_obj' (with 72.5% of missing values))
features_bins_woe = utils_ml.exclude_words(features_bins_woe, ["score","o_obj"])
features_bins_woe

['n_boolean_bins',
 'p_boolean_bins',
 'categoria_produto_bins',
 'pais_bins',
 'a_int_bins',
 'b_float_bins',
 'c_float_bins',
 'd_float_bins',
 'e_float_bins',
 'f_float_bins',
 'h_float_bins',
 'k_float_bins',
 'l_float_bins',
 'm_float_bins',
 'monto_bins']

# Model

## Logistic regression

In [70]:
df_2_model = df[features_bins_woe + ["fraude","data"]]

In [71]:
# df_2_model

### Dumming features

In [72]:
# from sklearn import preprocessing

# le = preprocessing.LabelEncoder()

# cat_vars = features_bins_woe

# df_2_model["pais_hot_encode"] = le.fit_transform(df_2_model['pais_bins'])


# X = data.iloc[:, 0:4]
# y = data.iloc[:, 5]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

# classifier = LogisticRegression()
# classifier.fit(X_train, y_train)

# df_2_model[["pais_hot_encode",'pais_bins']].value_counts()

In [73]:
def transform_2_dummies( df , label = "fraude", drop_first = True):

    features = utils_ml.list_subset_words( df.columns.tolist(), ["bin"])
    features = utils_ml.exclude_words(features, ["score","o_obj","data"])

    ls_df_dummies = []

    for var in features:

        ## creating dummy to variable
        df_dummy = pd.get_dummies( df[var] , prefix=var, drop_first = drop_first)
        ls_df_dummies.append(df_dummy)

    try:
        ## Quando exista o label e o campo data
        df_2_model_dum = pd.concat(ls_df_dummies + [ df[[label,'data']] ], axis=1)
    except:
        try:
            ## Quando exista o campo label
            df_2_model_dum = pd.concat(ls_df_dummies + [ df[[label]] ], axis=1)
        except:
            ## Quando não exista o campo label nem o campo data
            df_2_model_dum = pd.concat(ls_df_dummies , axis=1)


    return df_2_model_dum

In [74]:
df_2_model_dum = transform_2_dummies( df_2_model ,  label = "fraude")

In [75]:
def transform_2_model(df ,  label ="fraude", cols_model = utils_ml.exclude_words( df_2_model_dum.columns.tolist(),['data']) ):

    # print(cols_model)

    ## tirando a coluna label caso esteja na lista
    cols_model = utils_ml.exclude_words(cols_model, [label] )
    ## Transformando para dummies
    df_marco_model_dum = transform_2_dummies(df = df, label ="fraude")

    ## Preenchendo colunas faltantes
    dummies_faltantes = list(   set(cols_model) - set(df_marco_model_dum.columns.to_list()) )

    if len(dummies_faltantes)>0:

        for dummi_falt in dummies_faltantes:

            df_marco_model_dum[dummi_falt] = 0

    try:
        ## Quando o label exista no dataFrame
        df_2_model = df_marco_model_dum[cols_model+[label]]

    except:
        ## Quando não exista o campo label
        df_2_model = df_marco_model_dum[cols_model]


    return df_2_model

### Train and test samples

In [76]:
label = "fraude"

X_train, X_test, y_train, y_test = metrics_evaluation.train_test_split_2_model( df_2_model_dum, label, test_size = 0.3, random_state = 0, by_date = False, date_field = "data", n_days_test = 5 )

In [77]:
X_train.shape[0]/df_2_model_dum.shape[0]

0.6999992272801032

>> Weigth

In [78]:
## balance weight
w1_balanced = X_train.shape[0]/(2*y_train.sum())
w0_balanced = X_train.shape[0]/( 2*((y_train + 1)%2).sum() )
w1_balanced, w0_balanced

(9.863784843205575, 0.5266986057653174)

In [79]:
## weigth ratio
ratio = ((y_train + 1)%2).sum() / y_train.sum()

w0 = 1
w1 = ratio if ratio == int( ratio ) else int(ratio) +1

w1, w0

(19, 1)

In [80]:
y_train.sum()/X_train.shape[0] , ((y_train + 1)%2).sum() /X_train.shape[0]

(0.05069048118424974, 0.9493095188157502)

In [81]:
42617/43400, 783/43400

(0.9819585253456221, 0.01804147465437788)

### Trainning

In [82]:
2*3*5

30

In [83]:
lr = LogisticRegression(random_state=0, solver='liblinear', fit_intercept=True, penalty="l1" )


# #Creating a dictionary grid for grid searh
param_grid = { 'class_weight': [{0:1, 1:1},{0:w0, 1:w1}, {0:w0_balanced, 1:w1_balanced}] }

## Fitting grid search to the train data with 5 folds
clf_cv = GridSearchCV( estimator = lr,
                           param_grid = param_grid,
                           cv = StratifiedKFold(n_splits = 5), 
                           n_jobs=-1, 
                           scoring='recall', 
                           verbose=2 )


# clf_best = clf_cv.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


In [85]:
# clf_best.best_params_['class_weight']

In [24]:
# model_best_param = clf_best.best_params_
# model_best_param['random_state'] = 0
# model_best_param['solver'] = 'liblinear'
# model_best_param['fit_intercept'] = True
# model_best_param['penalty'] ="l1"


# with open( os.path.join(parameters.model_path, f"best_params_lr.json") , "w" ) as outfile:
#     json.dump(model_best_param, outfile)

# model_best_param

{'class_weight': {0: 1, 1: 19},
 'max_iter': 100,
 'random_state': 0,
 'solver': 'liblinear',
 'fit_intercept': True,
 'penalty': 'l1'}

>> **Best model**

In [25]:
clf = LogisticRegression( class_weight = {0: 1, 1: 19},
                    max_iter = 500,
                    random_state = 0,
                    solver = 'liblinear',
                    fit_intercept = False,
                    penalty = 'l1' )

model = clf.fit(X_train, y_train )

In [40]:
colunas_orig = utils_ml.exclude_words( df.columns.tolist(), ["bin","cat","WoE","fecha",'data','hora'] ) + ['categoria_produto']
colunas_orig, len(colunas_orig)

(['a_int',
  'b_float',
  'c_float',
  'd_float',
  'e_float',
  'f_float',
  'pais',
  'h_float',
  'produto',
  'k_float',
  'l_float',
  'm_float',
  'n_boolean',
  'o_obj',
  'p_boolean',
  'monto',
  'score',
  'fraude',
  'categoria_produto'],
 19)

In [41]:
df_ivs_ls = []

for variable in colunas_orig:

    label = "fraude"
    dtype_ = 'categorical' if df[variable].dtype=='O' or df[variable].dtype=='category' else 'numerical'

    df_iv = utils_ml.WOE_IV(df = df , variable = variable, dtype_ = dtype_ , label = label, monotonic= True, optimal_binning=True)[[ 'variable' , 'IV_var', 'KS']].drop_duplicates()

    df_ivs_ls.append(df_iv)

pd.concat(df_ivs_ls).sort_values("IV_var", ascending=False)

,variable,IV_var,KS
0,produto,6.575706,89.262752
0,categoria_produto,1.541493,40.920152
0,o_obj,1.111551,45.338385
0,score,0.989984,42.673514
0,f_float,0.443467,27.653106
0,l_float,0.428472,25.054252
0,n_boolean,0.363338,22.847129
0,m_float,0.341464,23.071558
0,p_boolean,0.246970,24.429978
0,d_float,0.170680,16.498234


In [42]:
coef_df = pd.DataFrame( {"Cols": model.feature_names_in_,  "coef": model.coef_[0] })
coef_df.sort_values("coef", ascending=False).reset_index(drop=True)

,Cols,coef
0,categoria_produto_bins_10,4.309233
1,categoria_produto_bins_9,3.758460
2,categoria_produto_bins_8,3.458901
3,categoria_produto_bins_7,3.313664
4,categoria_produto_bins_6,3.146327
5,categoria_produto_bins_5,3.016654
6,categoria_produto_bins_4,2.889257
7,categoria_produto_bins_3,2.720975
8,categoria_produto_bins_2,2.615266
9,categoria_produto_bins_1,2.376407


In [43]:
df_2_model_dum['probability'] = model.predict_proba(df_2_model_dum.drop(['fraude','data'], axis=1))[:,1]

## Catboost

In [44]:
import catboost as cb

### Train and test samples

In [45]:
def transform_2_cat_model(df):

    cat_features = [x for x in utils_ml.list_subset_words( df.columns.tolist(), ['cat']) if "WoE" not in x and "bin" not in x and x!='categoria_produto']
    cat_features = utils_ml.exclude_words(cat_features, ["score","o_obj","data"])

    try:
        df_2_model_cat = df[ cat_features + ["fraude"] ]
    except:
        df_2_model_cat = df[ cat_features ]


    return df_2_model_cat

In [46]:
cat_features = [x for x in utils_ml.list_subset_words( df.columns.tolist(), ['cat']) if "WoE" not in x and "bin" not in x and x!='categoria_produto']
## Excluindo os campos de data, score e o_obj
cat_features = utils_ml.exclude_words(cat_features, ["score","o_obj","data"])


df_2_model_cat = df[cat_features + ["fraude","data"]]

label = "fraude"

X_train_cat, X_test_cat, y_train_cat, y_test_cat = metrics_evaluation.train_test_split_2_model( df_2_model_cat, label, test_size = 0.3, random_state = 0, by_date = False, date_field = "data", n_days_test =5 )

In [47]:
cat_features

['n_boolean_cat',
 'p_boolean_cat',
 'categoria_produto_cat',
 'pais_cat',
 'a_int_cat',
 'b_float_cat',
 'c_float_cat',
 'd_float_cat',
 'e_float_cat',
 'f_float_cat',
 'h_float_cat',
 'k_float_cat',
 'l_float_cat',
 'm_float_cat',
 'monto_cat']

### Trainning

In [33]:
2*3*3*4

72

In [48]:
cat = cb.CatBoostClassifier( random_seed=0, verbose=False, task_type='GPU', thread_count=6 )


# #Creating a dictionary grid for grid search
param_grid_cat = {'class_weights': [{0:w0, 1:w1}, {0:w0_balanced, 1:w1_balanced}],
             "l2_leaf_reg":[ 3.0 , 4.0 , 5.0],
             "depth":[ 2, 3, 6 ]}


## Fitting grid search to the train data with 5 folds
clf_cat = GridSearchCV(     estimator = cat, 
                           param_grid = param_grid_cat,
                           cv = StratifiedKFold(n_splits = 4),
                           scoring = 'recall', 
                           verbose = 2 
                    )


# clf_cat_best = clf_cat.fit(X_train_cat, y_train_cat, cat_features =  cat_features)

In [49]:
# clf_cat_best.best_score_

In [51]:
# model_best_param_cat = clf_cat_best.best_params_
# model_best_param_cat['random_seed'] = 0
# model_best_param_cat['verbose'] = False
# model_best_param_cat['task_type'] = "GPU"
# model_best_param_cat['thread_count'] = 6


# with open( os.path.join(parameters.model_path, f"best_params_cat.json") , "w" ) as outfile:
#     json.dump(model_best_param_cat, outfile)

# model_best_param_cat

>> **Best model**

In [52]:
cat = cb.CatBoostClassifier(    random_seed = 0, 
                                verbose = False,
                                task_type = 'GPU', 
                                thread_count = 6,
                                class_weights = {0: 1, 1: 19},
                                depth = 2,
                                l2_leaf_reg = 3.0,
                                )

modelo_cat = cat.fit(X_train_cat, y_train_cat, cat_features =  cat_features)

In [53]:
coef = pd.DataFrame( {"features":modelo_cat.feature_names_,"coef":modelo_cat.feature_importances_})
coef = coef.sort_values("coef", ascending=False).reset_index(drop=True).query("coef>0")
coef

,features,coef
0,categoria_produto_cat,71.145208
1,n_boolean_cat,7.126892
2,f_float_cat,4.027647
3,l_float_cat,3.434764
4,m_float_cat,3.170761
5,p_boolean_cat,2.454320
6,a_int_cat,1.772542
7,h_float_cat,1.673140
8,monto_cat,1.114990
9,b_float_cat,1.092971


In [54]:
# pd.concat(df_ivs_ls).sort_values("IV_var", ascending=False)

In [55]:
df_2_model_cat['probability'] = modelo_cat.predict_proba(df_2_model_cat.drop(['fraude','data'], axis=1))[:,1]

## Comparing with Meli model

> **DataSet Train_test**

In [56]:
label="fraude"
quantil=10

print(f"\t\t*************************************************************************************", )
print(f"\t\t*********************** DEV DATASET: TRAIN + TEST ************************************",)
print(f"\t\t*************************************************************************************", "\n\n")

print(f"\t\t========================== Modelo MELI =====================================")
display(utils_ml.metric_evaluation(df, 'score', label, quantil))
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df, label, 'score'))


print("\n\n",f"\t\t========================== Modelo RG =====================================")
display( utils_ml.metric_evaluation(df_2_model_dum, 'probability', label, quantil) )
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_2_model_dum, label, 'probability'))


print("\n\n",f"\t\t========================== Modelo Catboost =====================================")
display( utils_ml.metric_evaluation(df_2_model_cat, 'probability', label, quantil) )
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_2_model_cat, label, 'probability'))

		*************************************************************************************
		*********************** DEV DATASET: TRAIN + TEST ************************************
		************************************************************************************* 


		========================== Modelo MELI =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,BER,cost,F1,F2
quantil,,,,,,,,,,,,,,,,,,,
0,12942,"[0, 8]",368,12574,6560,122853,0,0,5.609756,10.234996,5.609756,10.234996,42.087631,1.000000,0.050690,0.500000,0.949310,0.096490,0.210725
1,12941,"[8, 18]",612,12329,6192,110279,12574,368,9.329268,10.035571,14.939024,20.270567,42.087631,0.943902,0.053163,0.476874,0.880584,0.100658,0.216942
2,12941,"[18, 28]",404,12537,5580,97950,24903,980,6.158537,10.204879,21.097561,30.475446,42.087631,0.850610,0.053897,0.473342,0.832606,0.101372,0.214996
3,12941,"[28, 39]",43,12898,5176,85413,37440,1384,0.655488,10.498726,21.753049,40.974172,42.087631,0.789024,0.057137,0.453111,0.766948,0.106558,0.221520
4,12942,"[39, 49]",89,12853,5133,72515,50338,1427,1.356707,10.462097,23.109756,51.436269,42.087631,0.782470,0.066106,0.403894,0.670605,0.121912,0.247045
5,12941,"[49, 59]",166,12775,5044,59662,63191,1516,2.530488,10.398606,25.640244,61.834876,42.087631,0.768902,0.077953,0.358367,0.578164,0.141554,0.277307
6,12941,"[59, 69]",289,12652,4878,46887,75966,1682,4.405488,10.298487,30.045732,72.133363,42.087631,0.743598,0.094234,0.319027,0.492277,0.167270,0.312672
7,12941,"[69, 79]",710,12231,4589,34235,88618,1971,10.823171,9.955801,40.868902,82.089163,42.087631,0.699543,0.118200,0.289562,0.416844,0.202230,0.352653
8,12941,"[79, 89]",1157,11784,3879,22004,100849,2681,17.637195,9.591951,58.506098,91.681115,42.087631,0.591311,0.149867,0.293899,0.377195,0.239127,0.372101


AUROC , AUPR:  (0.7244964088550802, 0.20112328214314518)


 		========================== Modelo RG =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,BER,cost,F1,F2
quantil,,,,,,,,,,,,,,,,,,,
0,12942,"[0.004678355027226659, 0.019561763739378242]",14,12928,6560,122853,0,0,0.213415,10.523146,0.213415,10.523146,50.646483,1.000000,0.050690,0.500000,0.949310,0.096490,0.210725
1,12941,"[0.01956234901366563, 0.036143620312851864]",22,12919,6546,109925,12928,14,0.335366,10.515820,0.548780,21.038965,50.646483,0.997866,0.056203,0.448451,0.850494,0.106412,0.229345
2,12941,"[0.036148264839974756, 0.1293786669509601]",35,12906,6524,97006,25847,36,0.533537,10.505238,1.082317,31.544203,50.646483,0.994512,0.063016,0.397549,0.752366,0.118521,0.251368
3,12941,"[0.1293827435974006, 0.23957790261495784]",191,12750,6489,84100,38753,71,2.911585,10.378257,3.993902,41.922460,50.646483,0.989177,0.071631,0.347691,0.655344,0.133589,0.277714
4,12942,"[0.23958163661298448, 0.32592100174377275]",262,12680,6298,71350,51503,262,3.993902,10.321278,7.987805,52.243738,50.646483,0.960061,0.081110,0.310357,0.571581,0.149582,0.303115
5,12941,"[0.3259240677928636, 0.4100807045328598]",386,12555,6036,58670,64183,524,5.884146,10.219531,13.871951,62.463269,50.646483,0.920122,0.093283,0.278720,0.493845,0.169394,0.331845
6,12941,"[0.41008125466515233, 0.5010160936650703]",528,12413,5650,46115,76738,910,8.048780,10.103945,21.920732,72.567214,50.646483,0.861280,0.109147,0.257043,0.426657,0.193742,0.362156
7,12941,"[0.5010201303418187, 0.6045170376467361]",800,12141,5122,33702,89151,1438,12.195122,9.882543,34.115854,82.449757,50.646483,0.780793,0.131929,0.246768,0.371539,0.225718,0.393612
8,12941,"[0.6045233525016737, 0.7395491216296509]",1173,11768,4322,21561,101292,2238,17.881098,9.578928,51.996951,92.028685,50.646483,0.658841,0.166982,0.258330,0.339541,0.266437,0.414596


AUROC , AUPR:  (0.8397768939053276, 0.27067979319244034)


 		========================== Modelo Catboost =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,BER,cost,F1,F2
quantil,,,,,,,,,,,,,,,,,,,
0,12942,"[0.006888779513001193, 0.025412080627556143]",11,12931,6560,122853,0,0,0.167683,10.525587,0.167683,10.525587,50.855235,1.000000,0.050690,0.500000,0.949310,0.096490,0.210725
1,12941,"[0.025412141265491118, 0.041348811272829265]",21,12920,6549,109922,12931,11,0.320122,10.516634,0.487805,21.042221,50.855235,0.998323,0.056229,0.448210,0.850239,0.106461,0.229450
2,12941,"[0.04135190298955501, 0.1384992992795406]",31,12910,6528,97002,25851,32,0.472561,10.508494,0.960366,31.550715,50.855235,0.995122,0.063054,0.397228,0.752026,0.118594,0.251522
3,12941,"[0.13850824602175124, 0.23711581900750514]",189,12752,6497,84092,38761,63,2.881098,10.379885,3.841463,41.930600,50.855235,0.990396,0.071720,0.347048,0.654664,0.133753,0.278056
4,12942,"[0.23711947728615043, 0.3095108292436325]",266,12676,6308,71340,51513,252,4.054878,10.318022,7.896341,52.248622,50.855235,0.961585,0.081238,0.309554,0.570731,0.149819,0.303596
5,12941,"[0.3095141665081612, 0.3854841904237031]",361,12580,6042,58664,64189,518,5.503049,10.239880,13.399390,62.488503,50.855235,0.921037,0.093376,0.278239,0.493335,0.169562,0.332175
6,12941,"[0.3854864230172138, 0.4694676721268897]",546,12395,5681,46084,76769,879,8.323171,10.089294,21.722561,72.577796,50.855235,0.866006,0.109746,0.254554,0.424022,0.194805,0.364143
7,12941,"[0.4694689851383921, 0.5692233235565096]",753,12188,5135,33689,89164,1425,11.478659,9.920800,33.201220,82.498596,50.855235,0.782774,0.132264,0.245724,0.370434,0.226291,0.394611
8,12941,"[0.5692248059815979, 0.7065413251102587]",1201,11740,4382,21501,101352,2178,18.307927,9.556136,51.509146,92.054732,50.855235,0.667988,0.169300,0.253513,0.334441,0.270135,0.420352


AUROC , AUPR:  (0.8427721073748062, 0.2996078215512456)


>> **test sample**

In [57]:
label="fraude"
quantil=10

print(f"\t\t*************************************************************************************", )
print(f"\t\t*********************************** TEST DATASET ************************************",)
print(f"\t\t*************************************************************************************","\n\n")


print(f"\t\t========================== Modelo MELI =====================================")
test_2_meli = X_test_cat[['monto_cat']].merge( df[['fraude','score']], left_index=True, right_index=True ).drop(['monto_cat'], axis =1)

display(utils_ml.metric_evaluation( test_2_meli, 'score', label, quantil))
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR( test_2_meli, label, 'score'))


print("\n\n",f"\t\t========================== Modelo RG =====================================")
df_test = pd.DataFrame()
df_test['fraude'] = y_test
df_test['probability'] = model.predict_proba(X_test)[:,1]

display( utils_ml.metric_evaluation(df_test, 'probability', label, quantil) )
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_test, label, 'probability'))


print("\n\n",f"\t\t========================== Modelo Catboost =====================================")
df_test_cat = pd.DataFrame()
df_test_cat['fraude'] = y_test_cat
df_test_cat['probability'] = modelo_cat.predict_proba(X_test_cat)[:,1]


display( utils_ml.metric_evaluation(df_test_cat, 'probability', label, quantil) )
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_test_cat, label, 'probability'))

		*************************************************************************************
		*********************************** TEST DATASET ************************************
		************************************************************************************* 


		========================== Modelo MELI =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,BER,cost,F1,F2
quantil,,,,,,,,,,,,,,,,,,,
0,3883,"[0, 8]",110,3773,1968,36856,0,0,5.589431,10.237139,5.589431,10.237139,42.853675,1.000000,0.050690,0.500000,0.949310,0.096490,0.210725
1,3882,"[8, 18]",187,3695,1858,33083,3773,110,9.502033,10.025505,15.091463,20.262644,42.853675,0.944106,0.053175,0.476761,0.880461,0.100680,0.216990
2,3882,"[18, 29]",105,3777,1671,29388,7468,297,5.335366,10.247992,20.426829,30.510636,42.853675,0.849085,0.053801,0.474144,0.833454,0.101190,0.214610
3,3883,"[29, 38]",7,3876,1566,25611,11245,402,0.355691,10.516605,20.782520,41.027241,42.853675,0.795732,0.057622,0.449581,0.763213,0.107463,0.223402
4,3882,"[38, 49]",32,3850,1559,21735,15121,409,1.626016,10.446060,22.408537,51.473301,42.853675,0.792175,0.066927,0.398776,0.665181,0.123426,0.250112
5,3882,"[49, 59]",52,3830,1527,17885,18971,441,2.642276,10.391795,25.050813,61.865097,42.853675,0.775915,0.078663,0.354676,0.574258,0.142844,0.279834
6,3883,"[59, 69]",84,3799,1475,14055,22801,493,4.268293,10.307684,29.319106,72.172781,42.853675,0.749492,0.094977,0.315929,0.489002,0.168591,0.315144
7,3882,"[69, 79]",230,3652,1391,10256,26600,577,11.686992,9.908834,41.006098,82.081615,42.853675,0.706809,0.119430,0.285732,0.412786,0.204333,0.356319
8,3882,"[79, 89]",347,3535,1161,6604,30252,807,17.632114,9.591383,58.638211,91.672998,42.853675,0.589939,0.149517,0.294622,0.377962,0.238570,0.371235


AUROC , AUPR:  (0.7286720753236944, 0.20188606836738154)


 		========================== Modelo RG =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,BER,cost,F1,F2
quantil,,,,,,,,,,,,,,,,,,,
0,3883,"[0.00481147918509103, 0.01974061398290416]",5,3878,1968,36856,0,0,0.254065,10.522032,0.254065,10.522032,50.66851,1.000000,0.050690,0.500000,0.949310,0.096490,0.210725
1,3882,"[0.019743970184748702, 0.036651474904989746]",5,3877,1963,32978,3878,5,0.254065,10.519318,0.508130,21.041350,50.66851,0.997459,0.056180,0.448660,0.850711,0.106370,0.229253
2,3882,"[0.03667599082881174, 0.13113324455192374]",11,3871,1958,29101,7755,10,0.558943,10.503039,1.067073,31.544389,50.66851,0.994919,0.063041,0.397334,0.752138,0.118570,0.251471
3,3883,"[0.13129781208133498, 0.24087987618801862]",66,3817,1947,25230,11626,21,3.353659,10.356523,4.420732,41.900912,50.66851,0.989329,0.071641,0.347613,0.655265,0.133608,0.277754
4,3882,"[0.2408820534507413, 0.3259798061251072]",81,3801,1881,21413,15443,87,4.115854,10.313110,8.536585,52.214022,50.66851,0.955793,0.080750,0.312599,0.573949,0.148919,0.301771
5,3882,"[0.3259819976411104, 0.4100499160189516]",112,3770,1800,17612,19244,168,5.691057,10.228999,14.227642,62.443021,50.66851,0.914634,0.092726,0.281613,0.496909,0.168382,0.329864
6,3883,"[0.4100555331400512, 0.5007119852502965]",151,3732,1688,13842,23014,280,7.672764,10.125895,21.900407,72.568917,50.66851,0.857724,0.108693,0.258923,0.428652,0.192936,0.360653
7,3882,"[0.5007534674701992, 0.6030040088411718]",230,3652,1537,10110,26746,431,11.686992,9.908834,33.587398,82.477751,50.66851,0.780996,0.131965,0.246657,0.371420,0.225780,0.393719
8,3882,"[0.60308680524225, 0.7408209090821924]",349,3533,1307,6458,30398,661,17.733740,9.585956,51.321138,92.063707,50.66851,0.664126,0.168319,0.255548,0.336596,0.268571,0.417919


AUROC , AUPR:  (0.8388774191050734, 0.2723911937769557)


 		========================== Modelo Catboost =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,BER,cost,F1,F2
quantil,,,,,,,,,,,,,,,,,,,
0,3883,"[0.006888779513001193, 0.025613583621506435]",3,3880,1968,36856,0,0,0.152439,10.527458,0.152439,10.527458,50.2403,1.000000,0.050690,0.500000,0.949310,0.096490,0.210725
1,3882,"[0.025615512524622634, 0.041821809858199534]",7,3875,1965,32976,3880,3,0.355691,10.513892,0.508130,21.041350,50.2403,0.998476,0.056238,0.448125,0.850144,0.106478,0.229486
2,3882,"[0.04182897253057777, 0.1402841582483184]",8,3874,1958,29101,7755,10,0.406504,10.511179,0.914634,31.552529,50.2403,0.994919,0.063041,0.397334,0.752138,0.118570,0.251471
3,3883,"[0.14034414243364088, 0.23789457964765778]",70,3813,1950,25227,11629,18,3.556911,10.345670,4.471545,41.898198,50.2403,0.990854,0.071752,0.346811,0.654415,0.133814,0.278182
4,3882,"[0.23790291997891166, 0.3106048194427944]",88,3794,1880,21414,15442,88,4.471545,10.294118,8.943089,52.192316,50.2403,0.955285,0.080707,0.312867,0.574232,0.148840,0.301611
5,3882,"[0.310607300036396, 0.385264537646668]",103,3779,1792,17620,19236,176,5.233740,10.253419,14.176829,62.445735,50.2403,0.910569,0.092314,0.283754,0.499176,0.167633,0.328398
6,3883,"[0.3852706551755146, 0.4687944190011725]",160,3723,1689,13841,23015,279,8.130081,10.101476,22.306911,72.547211,50.2403,0.858232,0.108757,0.258655,0.428369,0.193051,0.360867
7,3882,"[0.4688031572017808, 0.5678271259638132]",218,3664,1529,10118,26738,439,11.077236,9.941394,33.384146,82.488604,50.2403,0.776931,0.131278,0.248798,0.373686,0.224605,0.391670
8,3882,"[0.5678544179172796, 0.7071310924841868]",345,3537,1311,6454,30402,657,17.530488,9.596809,50.914634,92.085414,50.2403,0.666159,0.168835,0.254478,0.335463,0.269393,0.419198


AUROC , AUPR:  (0.839449692750604, 0.29566619562966234)


>> **validation sample**

In [65]:
label="fraude"
quantil=10


print(f"\t\t*************************************************************************************", )
print(f"\t\t****************************** VALIDATION DATASET ************************************",)
print(f"\t\t*************************************************************************************",)

print("\n\n",f"\t\t========================= Modelo MELI =====================================")
display(utils_ml.metric_evaluation(df_validation, 'score', label, quantil))
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_validation, label, 'score'))



print("\n\n",f"\t\t========================== Modelo RG =====================================")
df_validation_2_model = transform_2_model(df_validation)
df_validation_2_model['probability'] = model.predict_proba( df_validation_2_model.drop(['fraude'], axis=1 ) )[:,1]

display( utils_ml.metric_evaluation( df_validation_2_model, 'probability', label, quantil ) )
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_validation_2_model, label, 'probability'))

print("\n\n",f"\t\t========================== Modelo Catboost =====================================")

df_validation_2_cat_model = transform_2_cat_model(df_validation)
df_validation_2_cat_model['probability'] = modelo_cat.predict_proba(df_validation_2_cat_model )[:,1]


display( utils_ml.metric_evaluation(df_validation_2_cat_model, 'probability', label, quantil) )
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_validation_2_cat_model, label, 'probability'))

		*************************************************************************************
		****************************** VALIDATION DATASET ************************************
		*************************************************************************************


 		========================= Modelo MELI =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,BER,cost,F1,F2
quantil,,,,,,,,,,,,,,,,,,,
0,2059,"[0, 7]",53,2006,940,19647,0,0,5.638298,10.210210,5.638298,10.210210,45.812273,1.000000,0.045660,0.500000,0.954340,0.087332,0.193042
1,2059,"[7, 16]",71,1988,887,17641,2006,53,7.553191,10.118593,13.191489,20.328803,45.812273,0.943617,0.047873,0.477140,0.882644,0.091124,0.198986
2,2058,"[16, 25]",91,1967,816,15653,3994,124,9.680851,10.011707,22.872340,30.340510,45.812273,0.868085,0.049548,0.464313,0.820566,0.093745,0.201691
3,2059,"[25, 34]",7,2052,725,13686,5961,215,0.744681,10.444343,23.617021,40.784853,45.812273,0.771277,0.050309,0.462659,0.769223,0.094456,0.199494
4,2059,"[34, 44]",11,2048,718,11634,8013,222,1.170213,10.423983,24.787234,51.208836,45.812273,0.763830,0.058128,0.414161,0.672949,0.108035,0.222815
5,2058,"[44, 53]",19,2039,707,9586,10061,233,2.021277,10.378175,26.808511,61.587011,45.812273,0.752128,0.068687,0.367892,0.578812,0.125879,0.251548
6,2059,"[53, 62]",29,2030,688,7547,12100,252,3.085106,10.332366,29.893617,71.919377,45.812273,0.731915,0.083546,0.326107,0.488998,0.149973,0.286786
7,2058,"[62, 72]",60,1998,659,5517,14130,281,6.382979,10.169492,36.276596,82.088869,45.812273,0.701064,0.106703,0.289871,0.404479,0.185216,0.331622
8,2059,"[72, 84]",128,1931,599,3519,16128,341,13.617021,9.828473,49.893617,91.917341,45.812273,0.637234,0.145459,0.270939,0.336572,0.236853,0.380173


AUROC , AUPR:  (0.7357432351211651, 0.25313615106229714)


 		========================== Modelo RG =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,BER,cost,F1,F2
quantil,,,,,,,,,,,,,,,,,,,
0,2059,"[0.003910827116968831, 0.019129011206975605]",32,2027,940,19647,0,0,3.404255,10.317097,3.404255,10.317097,34.553519,1.000000,0.045660,0.500000,0.954340,0.087332,0.193042
1,2059,"[0.01915223175258967, 0.03523252962451793]",45,2014,908,17620,2027,32,4.787234,10.250929,8.191489,20.568026,34.553519,0.965957,0.049007,0.465436,0.871424,0.093281,0.203697
2,2058,"[0.03525447616015475, 0.13457780223491989]",63,1995,863,15606,4041,77,6.702128,10.154222,14.893617,30.722248,34.553519,0.918085,0.052401,0.438117,0.795453,0.099144,0.213308
3,2059,"[0.13458218562134072, 0.23004688141371274]",45,2014,800,13611,6036,140,4.787234,10.250929,19.680851,40.973177,34.553519,0.851064,0.055513,0.420857,0.729149,0.104228,0.220131
4,2059,"[0.2300682837539878, 0.3134156594123464]",44,2015,755,11597,8050,185,4.680851,10.256019,24.361702,51.229195,34.553519,0.803191,0.061124,0.393538,0.653179,0.113602,0.234297
5,2058,"[0.31343207642208115, 0.40255414186583016]",55,2003,711,9582,10065,229,5.851064,10.194941,30.212766,61.424136,34.553519,0.756383,0.069076,0.365663,0.576675,0.126591,0.252971
6,2059,"[0.4026861879276317, 0.5059227203441496]",71,1988,656,7579,12068,284,7.553191,10.118593,37.765957,71.542729,34.553519,0.697872,0.079660,0.343943,0.506096,0.142997,0.273447
7,2058,"[0.505959043301805, 0.6431319846435491]",87,1971,585,5591,14056,355,9.255319,10.032066,47.021277,81.574795,34.553519,0.622340,0.094722,0.331116,0.444018,0.164418,0.294384
8,2059,"[0.643179154196612, 0.8096951664935497]",138,1921,498,3620,16027,442,14.680851,9.777574,61.702128,91.352369,34.553519,0.529787,0.120932,0.327232,0.390538,0.196916,0.316070


AUROC , AUPR:  (0.7194033467293475, 0.18473295804508308)


 		========================== Modelo Catboost =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,BER,cost,F1,F2
quantil,,,,,,,,,,,,,,,,,,,
0,2059,"[0.0076466491092422465, 0.02602610147394066]",26,2033,940,19647,0,0,2.765957,10.347636,2.765957,10.347636,34.664991,1.000000,0.045660,0.500000,0.954340,0.087332,0.193042
1,2059,"[0.02604058560418861, 0.04205854112316137]",45,2014,914,17614,2033,26,4.787234,10.250929,7.553191,20.598565,34.664991,0.972340,0.049331,0.462092,0.868218,0.093898,0.205043
2,2058,"[0.042059412883842205, 0.13484677607978346]",88,1970,869,15600,4047,71,9.361702,10.026976,16.914894,30.625541,34.664991,0.924468,0.052766,0.434773,0.792248,0.099833,0.214791
3,2059,"[0.13489035508907007, 0.2275947012159182]",52,2007,781,13630,6017,159,5.531915,10.215300,22.446809,40.840841,34.664991,0.830851,0.054195,0.431447,0.739302,0.101752,0.214903
4,2059,"[0.22762809865448277, 0.3000670121289924]",36,2023,729,11623,8024,211,3.829787,10.296737,26.276596,51.137578,34.664991,0.775532,0.059019,0.408030,0.667071,0.109690,0.226229
5,2058,"[0.30006910296633044, 0.37523792894092345]",45,2013,693,9600,10047,247,4.787234,10.245839,31.063830,61.383417,34.664991,0.737234,0.067327,0.375695,0.586292,0.123386,0.246567
6,2059,"[0.3752418879671063, 0.45339809724150143]",57,2002,648,7587,12060,292,6.063830,10.189851,37.127660,71.573268,34.664991,0.689362,0.078689,0.348402,0.510371,0.141253,0.270113
7,2058,"[0.4534565592983629, 0.5543840637297086]",92,1966,591,5585,14062,349,9.787234,10.006617,46.914894,81.579885,34.664991,0.628723,0.095693,0.327772,0.440812,0.166105,0.297403
8,2059,"[0.5545439201262801, 0.6916477937812148]",130,1929,499,3619,16028,441,13.829787,9.818293,60.744681,91.398178,34.664991,0.530851,0.121175,0.326675,0.390003,0.197311,0.316705


AUROC , AUPR:  (0.7169783108026888, 0.20627995367467755)


## Using period in evaluation

> **modelo MELI**

Apresenta uma relativa estabilidade quando se olha os score para as semanas

In [86]:
df_full = pd.concat([df,df_validation])

In [95]:
## Ordenando as datas disponíveis no dataset
datas = np.sort( df_full['data'].unique() )
## total de dias para cada grupo
days = 5
## Total de grupos
total_groups = int(len(datas)/days) if int(len(datas)/days) == len(datas)/days else int(len(datas)/days+1)
## Lista de arrays contendo os dias para cada grupo
ls_group = []

i = 0
for x in range(1,total_groups+1):

    # print(x)
    group = datas[i:i+days]
    ls_group.append(group)

    i = i+days

print(f"Total de grupos:", total_groups)

Total de grupos: 9


In [96]:
ls_group

[array(['2020-03-08T00:00:00.000000000', '2020-03-09T00:00:00.000000000',
        '2020-03-10T00:00:00.000000000', '2020-03-11T00:00:00.000000000',
        '2020-03-12T00:00:00.000000000'], dtype='datetime64[ns]'),
 array(['2020-03-13T00:00:00.000000000', '2020-03-14T00:00:00.000000000',
        '2020-03-15T00:00:00.000000000', '2020-03-16T00:00:00.000000000',
        '2020-03-17T00:00:00.000000000'], dtype='datetime64[ns]'),
 array(['2020-03-18T00:00:00.000000000', '2020-03-19T00:00:00.000000000',
        '2020-03-20T00:00:00.000000000', '2020-03-21T00:00:00.000000000',
        '2020-03-22T00:00:00.000000000'], dtype='datetime64[ns]'),
 array(['2020-03-23T00:00:00.000000000', '2020-03-24T00:00:00.000000000',
        '2020-03-25T00:00:00.000000000', '2020-03-26T00:00:00.000000000',
        '2020-03-27T00:00:00.000000000'], dtype='datetime64[ns]'),
 array(['2020-03-28T00:00:00.000000000', '2020-03-29T00:00:00.000000000',
        '2020-03-30T00:00:00.000000000', '2020-03-31T00:00:00.0000

In [97]:

label="fraude"
quantil=10

@interact

def meli_model( i  = range( len(ls_group) ) ):

    print(f"Grupo {i}: ", ls_group[i], "\n\n")


    print(f"========================== Modelo MELI =====================================")
    df_marco = df_full[df_full['data'].isin(ls_group[i])]

    display(utils_ml.metric_evaluation(df_marco, 'score', label, quantil))
    print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_marco, label, 'score'))

    print("\n\n",f"========================== Modelo RG =====================================")
    df_marco_model_dum = transform_2_model( df_full[df_full['data'].isin(ls_group[i])] )
    ######### Fitting ###########################
    df_marco_model_dum['probability'] = model.predict_proba( df_marco_model_dum.drop("fraude", axis=1) )[:,1]

    display(utils_ml.metric_evaluation(df_marco_model_dum, 'probability', label, quantil))
    print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_marco_model_dum, label, 'probability'))


    print("\n\n",f"========================== Modelo Catboost =====================================")
    df_marco_cat = transform_2_cat_model( df_full[df_full['data'].isin(ls_group[i])] )
    ######### Fitting ###########################
    df_marco_cat['probability'] = modelo_cat.predict_proba(df_marco_cat)[:,1]

    display(utils_ml.metric_evaluation(df_marco_cat, 'probability', label, quantil))
    print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_marco_cat, label, 'probability'))
    

interactive(children=(Dropdown(description='i', options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=0), Output()), _dom…

In [63]:
!lsb_release -a

'lsb_release' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


# 4. Instabilidade


- Supondo que o desempenho produtivo seja muito diferente do esperado, quais você acha que são as causas mais prováveis?

    1. Pode estar associado a sazonalidade, pode ser que o conjunto disponibilizado não contenha toda a informação necessária para o modelo performar em produção.
    1. Os Fraudadores cada vez usam novas estratégias tendo que mudar o ajustar o modelo a cada determinado tempo, portanto, pode ser que o modelo depois de um tempo perda sua performance e estabilidade.
    1. Aparição de novas features/caracteristicas dos fraudadores que não se tenham disponíveis, provacando perda de performance para novas safras.
    1. O comportamento de cada pais seja diferente, e dependendo, das novas safras, não consiga performar bem. Talvez seja melhor segmentar por pais.


# 5.Custom scorer


- Com a informação adicional de que uma fraude custa 10 vezes mais para a
empresa do que o bloqueio incorreto de um pagamento, o que você faria de
diferente?

    - É possível definir uma métrica personalizada de custo para ser usada no gridsearch procurando a melhor configuração do modelo
    


        \begin{align*}

        &\text{cost} =​ FP + 10*FN \\
        &\text{Recall\_cost} =​ \frac{​TP​​}{TP+10*FN} 

        \end{align*}

In [101]:
def custom_cost_metric (y_actual, y_pred):

    """
    Função personalizada para usar como métrica de seleção no treino dos modelos usando o Gridsearch
    
     """

    TN , FP , FN , TP = confusion_matrix(y_actual, y_pred).ravel().tolist()

    cost = FP + 10*FN

    return cost


def custom_recall_cost (y_actual, y_pred):

    """
    Função personalizada para usar como métrica de seleção no treino dos modelos usando o Gridsearch
    
     """

    TN , FP , FN , TP = confusion_matrix(y_actual, y_pred).ravel().tolist()

    recall_cost = TP/(TP+10*FN)

    return recall_cost

In [102]:
## sklearn make_scorer for the cost function
custom_cost_scorer = make_scorer( custom_cost_metric, greater_is_better = False)

## Recall modificada
custom_cost_scorer = make_scorer( custom_recall_cost, greater_is_better = False)

# Requirements

In [110]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt

5697.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
